## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [46]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

mtx=None
dist=None    
img_size = None

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_size = (gray.shape[0], gray.shape[1])

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)


    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], mtx, dist)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)



#cv2.destroyAllWindows()

In [174]:
# calculate points
def get_x(k,b,y):
    x = (y - b) / k
    return x

xl = get_x(-240/325,886,450)
xr = get_x(24/37,3,450)

print(xl)
print(xr)
#y=-240/325*x+886

590.4166666666666
689.125


In [181]:
import matplotlib.image as mpimg

#print(img_size)
img_width = img_size[0]
img_height = img_size[1]
#print(img_height)


selected_upper_width=20
selected_points = [[(img_width-selected_upper_width)/2,400],
                   [(img_width+selected_upper_width)/2,400],
                   [img_width-100,img_height],
                   [100,img_height]]

src = np.float32(selected_points)

dst = np.float32([[100,0],
                  [img_width-100,0], 
                  [img_width-100,img_height],                    
                  [100,img_height]])

#selected_points=[[547,480],[733,480],[1090,720],[190,720]]
selected_points=[[590,450],[689,450],[1090,720],[190,720]]
src=np.float32(selected_points)
dst = np.float32([[190,0],[1090,0],[1090,720],[190,720]])

# Given src and dst points, calculate the perspective transform matrix        
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
# Warp the image using OpenCV warpPerspective()
#warped = cv2.warpPerspective(undist, M, img_size)

## And so on and so forth...

In [8]:
#for fname in images:
#    img = cv2.imread(fname)
#    undist = cv2.undistort(img, mtx, dist, None, mtx)
#    cv2.imshow('img',img)
#    cv2.waitKey(500)
img = cv2.imread('../camera_cal/calibration1.jpg')
undist = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


In [218]:
def showOriginAndOutput(origin,output):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(origin)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(output)
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
def show3x2(a,b,c,d,e,f):
    f, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1, 6, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(a)
    ax1.set_title('Image 1', fontsize=20)
    ax2.imshow(b)
    ax2.set_title('Image 2', fontsize=20)
    ax3.imshow(c)
    ax3.set_title('Image 3', fontsize=20)
    ax4.imshow(d)
    ax4.set_title('Image 4', fontsize=20)
    ax5.imshow(e)
    ax5.set_title('Image 5', fontsize=20)
    #ax6.imshow(f)
    #ax6.set_title('Image 6', fontsize=50)

    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [247]:
def abs_sobel_thresh(gray, orient='x', thresh_min=0, thresh_max=255):
    x=0
    y=0
    if orient=='y':
        y=1
    else:
        x=1
    sobel = cv2.Sobel(gray, cv2.CV_64F, x, y)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return sxbinary

def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    mag = np.sqrt(sobelx**2+sobely**2)
    scaled_mag = np.uint8(255*mag/np.max(mag))
    binary_output = np.zeros_like(scaled_mag)
    binary_output[(scaled_mag >= mag_thresh[0]) & (scaled_mag <= mag_thresh[1])] = 1
    return binary_output
    

def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    absx = np.absolute(sobelx)
    absy = np.absolute(sobely)
    direction = np.arctan2(absy,absx)
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    return binary_output
    
def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    hls = cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    s_bin = abs_sobel_thresh(S, orient='x', thresh_min=20, thresh_max=100)
    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    combined = np.zeros_like(S)
    combined[(binary == 1) | (s_bin==1) ] = 1
    return combined
    
def getBinary(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    gradx = abs_sobel_thresh(gray, orient='x', thresh_min=20, thresh_max=100)
    grady = abs_sobel_thresh(gray, orient='y', thresh_min=20, thresh_max=100)
    mag_binary = mag_thresh(gray, sobel_kernel=3, mag_thresh=(30, 100))
    dir_binary = dir_threshold(gray, sobel_kernel=15, thresh=(0.7, 1.3))
    
    s_bin = abs_sobel_thresh(gray, orient='x', thresh_min=20, thresh_max=100)
    hls_binary = hls_select(img, thresh=(150, 225))

    combined = np.zeros_like(dir_binary)
    #combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (hls_binary==1) ] = 1
    
    combined[(gradx == 1) | (hls_binary==1) ] = 1
    
    #show3x2(gradx,grady,mag_binary,hls_binary,combined,combined)

    return combined


In [256]:
def find_lane_in_warped_bin(warped_bin, show=False):
    # Create a binary version of the warped image
    #warped_bin = np.zeros_like(warped[:,:,0])
    #warped_bin[(warped[:,:,0] > 0)] = 1
    
    #vis_img = warped.copy()  # The image we will draw on to show the lane-finding process
    #vis_img[vis_img > 0] = 255  # Max out non-black pixels so we can remove them later

    # Sum the columns in the bottom portion of the image to create a histogram
    histogram = np.sum(warped_bin[warped_bin.shape[0]//2:,:], axis=0)
    # Find the left an right right peaks of the histogram
    midpoint = histogram.shape[0]//2
    left_x = np.argmax(histogram[:midpoint])  # x-position for the left window
    right_x = np.argmax(histogram[midpoint:]) + midpoint  # x-position for the right window

    n_windows = 10
    win_height = warped_bin.shape[0]//n_windows
    margin = 80  # Determines how wide the window is
    pix_to_recenter = margin*2  # If we find this many pixels in our window we will recenter (too few would be a bad recenter)

    # Find the non-zero x and y indices
    nonzero_ind = warped_bin.nonzero()
    nonzero_y_ind = np.array(nonzero_ind[0])
    nonzero_x_ind = np.array(nonzero_ind[1])

    left_line_ind, right_line_ind = [], []

    for win_i in range(n_windows):
        win_y_low = warped_bin.shape[0] - (win_i+1)*win_height
        win_y_high = warped_bin.shape[0] - (win_i)*win_height
        win_x_left_low = max(0, left_x - margin)
        win_x_left_high = left_x + margin
        win_x_right_low = right_x - margin
        win_x_right_high = min(warped_bin.shape[1]-1, right_x + margin)

        # Draw the windows on the vis_img
        #rect_color, rect_thickness = (0, 255, 0), 3
        #cv2.rectangle(vis_img, (win_x_left_low, win_y_high), (win_x_left_high, win_y_low), rect_color, rect_thickness)
        #cv2.rectangle(vis_img, (win_x_right_low, win_y_high), (win_x_right_high, win_y_low), rect_color, rect_thickness)

        # Record the non-zero pixels within the windows
        left_ind = (
            (nonzero_y_ind >= win_y_low) &
            (nonzero_y_ind <= win_y_high) &
            (nonzero_x_ind >= win_x_left_low) &
            (nonzero_x_ind <= win_x_left_high)
        ).nonzero()[0]
        right_ind = (
            (nonzero_y_ind >= win_y_low) &
            (nonzero_y_ind <= win_y_high) &
            (nonzero_x_ind >= win_x_right_low) &
            (nonzero_x_ind <= win_x_right_high)
        ).nonzero()[0]
        left_line_ind.append(left_ind)
        right_line_ind.append(right_ind)

        # If there are enough pixels, re-align the window
        if len(left_ind) > pix_to_recenter:
            left_x = int(np.mean(nonzero_x_ind[left_ind]))
        if len(right_ind) > pix_to_recenter:
            right_x = int(np.mean(nonzero_x_ind[right_ind]))

    # Combine the arrays of line indices
    left_line_ind = np.concatenate(left_line_ind)
    right_line_ind = np.concatenate(right_line_ind)

    # Gather the final line pixel positions
    left_x = nonzero_x_ind[left_line_ind]
    left_y = nonzero_y_ind[left_line_ind]
    right_x = nonzero_x_ind[right_line_ind]
    right_y = nonzero_y_ind[right_line_ind]

    # Color the lines on the vis_img
    #vis_img[left_y, left_x] = [254, 0, 0]  # 254 so we can isolate the white 255 later
    #vis_img[right_y, right_x] = [0, 0, 254]  # 254 so we can isolate the white 255 later

    # Fit a 2nd-order polynomial to the lines
    left_fit = np.polyfit(left_y, left_x, 2)
    right_fit = np.polyfit(right_y, right_x, 2)

    # Get our x/y vals for the fit lines
    y_vals = np.linspace(0, warped_bin.shape[0]-1, warped_bin.shape[0])
    left_x_vals = left_fit[0]*y_vals**2 + left_fit[1]*y_vals + left_fit[2]
    right_x_vals = right_fit[0]*y_vals**2 + right_fit[1]*y_vals + right_fit[2]

    #if show:
    #    fig, ax = plt.subplots(figsize=(20, 10))
    #    ax.imshow(vis_img)
    #    ax.plot(left_x_vals, y_vals, color='yellow')
    #    ax.plot(right_x_vals, y_vals, color='yellow')
    #    cv2.imwrite('./example_images/lane_detection_warped_test2.jpg', vis_img)
        
    #lane_lines_img = vis_img.copy()
    #lane_lines_img[lane_lines_img == 255] = 0  # This basically removes everything except the colored lane lines
    
    return y_vals, left_x_vals, right_x_vals, left_fit, right_fit

#y_vals, left_x_vals, right_x_vals, left_fit, right_fit  = find_lane(warped, show=True)


In [271]:
def draw_lane(img, y_vals, left_x_vals, right_x_vals, show=False):
    # Prepare the x/y points for cv2.fillPoly()
    left_points = np.array([np.vstack([left_x_vals, y_vals]).T])
    right_points = np.array([np.flipud(np.vstack([right_x_vals, y_vals]).T)])
    # right_points = np.array([np.vstack([right_x_vals, y_vals]).T])
    points = np.hstack((left_points, right_points))

    # Color the area between the lines (the lane)
    lane = np.zeros_like(img)  # Create a blank canvas to draw the lane on
    cv2.fillPoly(lane, np.int_([points]), (0, 255, 0))
    #warped_lane_info = cv2.addWeighted(lane_lines_img, 1, lane, .3, 0)

    #unwarped_lane_info = cv2.warpPerspective(warped_lane_info, Minv, (img.shape[1], img.shape[0]))
    #drawn_img = cv2.addWeighted(img, 1, unwarped_lane_info, 1, 0)
    
    #if show: big_plot(drawn_img)
        
    return lane

#drawn_img = draw_lane(img, lane_lines_img, y_vals, left_x_vals, right_x_vals, show=True)

In [309]:
def pipeline(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #binary = getBinary(undist)
    warped = cv2.warpPerspective(undist, M, (img_size), flags=cv2.INTER_LINEAR)
    wraped_binary = getBinary(warped)
    
    vis_img = warped.copy()  # The image we will draw on to show the lane-finding process
    vis_img[vis_img > 0] = 255  # Max out non-black pixels so we can remove them later

    #histogram = np.sum(wraped_binary[wraped_binary.shape[0]//2:,:], axis=0) 

    #midpoint = histogram.shape[0]//2
    #left_x = np.argmax(histogram[:midpoint])  # x-position for the left window
    #right_x = np.argmax(histogram[midpoint:]) + midpoint  # x-position for the right window

    y_vals, left_x_vals, right_x_vals, left_fit, right_fit  = find_lane_in_warped_bin(wraped_binary, show=True)
    
    left_curverad = get_curve()
    right_curverad = 
    lane = draw_lane(img, y_vals, left_x_vals, right_x_vals, show=False)
    
    warped_lane_img = vis_img.copy()
    warped_lane_img[warped_lane_img == 255] = 0  # This basically removes everything except the colored lane lines

    warped_lane_img = cv2.addWeighted(warped_lane_img, 1, lane, .3, 0)
    unwarped_lane_img = cv2.warpPerspective(warped_lane_img, Minv, (img.shape[1], img.shape[0]))
    output_img = cv2.addWeighted(img, 1, unwarped_lane_img, 1, 0)

    fig, ax = plt.subplots(figsize=(20, 10))
    ax.imshow(output_img)
    ax.plot(left_x_vals, y_vals, color='yellow')
    ax.plot(right_x_vals, y_vals, color='yellow')

    #plt.plot(histogram)

    return output_img


In [310]:
#img = cv2.imread('../test_images/straight_lines2.jpg')
img = mpimg.imread('../test_images/test4.jpg')
#print(img.shape)
#origin = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
origin = img
#cv2.polylines(origin,[np.array(selected_points,np.int32)],True,(0,255,255))

output = pipeline(origin)
#showOriginAndOutput(origin,output)

In [305]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [307]:
f_name = 'challenge_video.mp4'
video = VideoFileClip('../{}'.format(f_name))  # Load the original video
video = video.fl_image(pipeline)  # Pipe the video frames through the lane-detection pipeline
# video = video.subclip(39, 43)  # Only process a portion
%time video.write_videofile('../output_videos/{}'.format(f_name), audio=False)  # Write the new video

[MoviePy] >>>> Building video ../output_videos/challenge_video.mp4
[MoviePy] Writing video ../output_videos/challenge_video.mp4



 21%|████████▎                               | 101/485 [00:47<02:25,  2.63it/s]


 42%|████████████████▋                       | 203/485 [01:41<04:12,  1.12it/s]


 63%|█████████████████████████▏              | 305/485 [02:36<01:08,  2.64it/s]


 84%|█████████████████████████████████▌      | 407/485 [03:17<00:39,  1.97it/s]


100%|████████████████████████████████████████| 485/485 [03:56<00:00,  1.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_videos/challenge_video.mp4 

Wall time: 3min 58s


In [325]:
bg = 1
gs_rate = 0.15
gs = bg*gs_rate
xf_rate = np.array([0.0,0.01,0.03,0.05,0.09,0.12,0.25,0.4])
xf = (bg+gs)/(1-xf_rate)*xf_rate
zz = (bg+gs+xf) * 0.17
sl = (gs+xf+zz) * 100
print(sl)

print(55/(1.5394736842))
print(35.726495726740005*1.4163157895)

[  34.55         35.90909091   38.71134021   41.63157895   47.85714286
   52.89772727   79.4         124.25      ]
35.726495726740005
50.60000000128615
